# Welcome to the Synergy Single Frame with Image Streamer POC
Created by Fredrik Tärnell - 2019-02-29<br>

Jupyter Notebooks can be found: https://github.com/frippe75/synergy-jupyter-notebooks<br>
Together with a Docker image with all dependencies inluded and a quickstart guide.


### API information

http://h17007.www1.hpe.com/docs/enterprise/servers/oneview4.1/cic-api/en/index.html <br>


### Python specific
https://hewlettpackard.github.io/python-hpOneView/index.html<br>
https://github.com/HewlettPackard/python-hpOneView<br>

## Table of content

- **[Settings](#Setting)**<br>
   + [Addresses and Indentifiers](#Address-identifiers)<br>
   + [Networks](#Networks)<br>
   + [Delete a single network](#Network-delete)<br>
   + [Create multiple networks](#Network-bulk-create)<br>
   + [Delete multiple networks](#Network-bulk-delete)<br>
   + [Delete ALL networks](#Network-delete-all)<br>
- **[Server Profiles](#SP)**<br>
   + [Power On/Off all Profiles](#SP-power)<br>
   + [Delete all Profiles](#SP-delete)<br>
- **[Server Profiles Templates](#SPT)**<br>
   + [Delete all Profiles Templates](#SPT-delete)<br>
- **[Appliances](#Appliances)**<br>
   + [eFuse Composer / Image Streamer](#Appliance-efuse)<br>

<br><br>
# Dependencies and logging in



### Import the python OneView library with some additional dependencies for this playbook

In [143]:
from hpOneView.oneview_client import OneViewClient
from hpOneView.exceptions import HPOneViewException
import pandas, yaml
from IPython.display import display, Markdown
from pprint import pprint

### Configure your environment
**Please note:** that if you use the option below to get the YAML doc from a file the config dict will be overwritten by the config from the file. During the login, check that you are logged in against the correct Oneview instance

In [150]:
composer_ip = "syn420-dcs.hpedemo.local"
config = {
    "api_version": "800",
    "ip": composer_ip,
    "credentials": {
        "userName": "Administrator",
        "authLoginDomain": "local",
        "password": "HPEc0nverged!"
    }
}

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.


### YAML file / document
Determine wheter to load the YAML doc from a variable or file<br>
Setting the filename takes precedence over variable

In [194]:
# Comment out to use the poc_file to allow the embedded poc_doc to be used
poc_file = "synergy-poc.yaml"

# Example YAML stucture for allowing Addresses and Indentifiers part work without a file
poc_doc = """
---
  addresses:
    # Synergy Management 
    - networkId: 10.48.0.0
      subnetmask: 255.255.255.0
      gateway: 10.48.0.1
      domain: hpedemo.local
      dnsServers: 
        - 10.48.10.4
        - 10.48.10.6
      range:
        name: LE-POC-01
        startAddress: 10.48.0.104
        endAddress: 10.48.0.124
    # Image Streamer iSCSI 
    - networkId: 192.168.10.0
      subnetmask: 255.255.255.0
      range:
        name: iSCSI-boot
        startAddress: 192.168.10.10
        endAddress: 192.168.10.100
    # VMware Mgmt 
    - networkId: 10.48.10.0
      subnetmask: 255.255.255.0
      gateway: 10.48.10.1
      domain: hpedemo.local
      dnsServers: 
        - 10.48.10.4
        - 10.48.10.6
      range:
        name: LE-POC-01
        startAddress: 10.48.10.70
        endAddress: 10.48.10.79
    # vMotion 
    - networkId: 192.168.20.0
      subnetmask: 255.255.255.0
      range:
        name: vMotion
        startAddress: 192.168.20.10
        endAddress: 192.168.20.100
"""
# load/parse the YAML doc into a variable
poc = ""
if poc_file is not None:
    with open(poc_file) as f:
        poc_doc = f.read()
    # print(poc_doc)
    poc = yaml.load(poc_doc)
    # Check if Oneview config key is part of the YAML file and use that for logging in.
    if 'config' in poc:
        print("\nOneview endpoint config was found and is used. Removing it to avoid displaying passwords\n")
        config = poc.pop('config', None)
else:
    poc = yaml.load(poc_doc)
    
# Print it out for sanitycheck
print (yaml.dump(poc))


Oneview endpoint config was found and is used. Removing it to avoid displaying passwords

addresses:
- dnsServers: [10.48.10.4, 10.48.10.6]
  domain: hpedemo.local
  gateway: 10.48.0.1
  networkId: 10.48.0.0
  range: {endAddress: 10.48.0.124, name: LE-POC-01, startAddress: 10.48.0.104}
  subnetmask: 255.255.255.0
- networkId: 192.168.10.0
  range: {endAddress: 192.168.10.100, name: iSCSI-boot, startAddress: 192.168.10.10}
  subnetmask: 255.255.255.0
- dnsServers: [10.48.10.4, 10.48.10.6]
  domain: hpedemo.local
  gateway: 10.48.10.1
  networkId: 10.48.10.0
  range: {endAddress: 10.48.10.79, name: LE-POC-01, startAddress: 10.48.10.70}
  subnetmask: 255.255.255.0
- networkId: 192.168.20.0
  range: {endAddress: 192.168.20.100, name: vMotion, startAddress: 192.168.20.10}
  subnetmask: 255.255.255.0
domain: hpedemo.local
networks:
- {association: 10.48.0.0, name: VLAN2000_10.48.0, purpose: Management, vlanId: 2000}



<br><br>
## Login against the Appliance / Composer instance:
Create new object "ov" with the config (see above) using the JSON config dict above

In [195]:
try:
    ov = OneViewClient(config)
    print("\nLogged in against " + config['ip'] + " successfully\n")
except HPOneViewException as e:
    print("\nLogged in failed:")
    print(e.msg)


Logged in against syn420-dcs.hpedemo.local successfully



<br><br>
# Settings  <a name="Settings"></a>
## Addresses and Indentifiers  <a name="Address-identifiers"></a>
Here we define required networks for both Synergy Management network as well as Image Streamer iSCSi boot network and VMware vMotion.

In [169]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

# Loop over the addresses array and create subnets and ranges
for options in poc['addresses']:
    
    options.update({
        "type": "Subnet",
        "name": "IPv4Subnet"
    })
    # Remove/extract  the range key since it does not belong there.
    range_option = options.pop('range', None)
    #pprint(options)  
    
    try:
        print("Create IPv4 subnet " + options['networkId'] +" for id pools")
        ipv4_subnet = ov.id_pools_ipv4_subnets.create(options)
        
        #pprint(range_option)
    
        range_option.update({
            "type": "Range",
            "rangeCategory": "Custom",
            "subnetUri": ipv4_subnet['uri']
        })
        print("\nCreating an IPv4 Range for id pools\n")
        ipv4_range = ov.id_pools_ipv4_ranges.create(range_option)
        # pprint(ipv4_range)
    except HPOneViewException as e:
        print(e.msg)
    
    
print("\nGetting all IPv4 subnets...")
all_subnets = ov.id_pools_ipv4_subnets.get_all()

#pprint(all_subnets)

data = pandas.DataFrame.from_dict(all_subnets)
table = data.to_html(index=False,header=True,columns=['networkId','subnetmask','domain','gateway'])

display(Markdown('<br>'))
display(Markdown('### Subnets in Synergy Composer at https://' + config['ip']))
display(Markdown(table))

Create IPv4 subnet 10.48.0.0 for id pools
The subnet for Network ID - 10.48.0.0 and Subnet Mask - 255.255.255.0 is overlapping with the subnet for Network ID - 10.48.0.0 and Subnet Mask - 255.255.255.0.
Create IPv4 subnet 192.168.10.0 for id pools
The subnet for Network ID - 192.168.10.0 and Subnet Mask - 255.255.255.0 is overlapping with the subnet for Network ID - 192.168.10.0 and Subnet Mask - 255.255.255.0.
Create IPv4 subnet 10.48.10.0 for id pools
The subnet for Network ID - 10.48.10.0 and Subnet Mask - 255.255.255.0 is overlapping with the subnet for Network ID - 10.48.10.0 and Subnet Mask - 255.255.255.0.
Create IPv4 subnet 192.168.20.0 for id pools
The subnet for Network ID - 192.168.20.0 and Subnet Mask - 255.255.255.0 is overlapping with the subnet for Network ID - 192.168.20.0 and Subnet Mask - 255.255.255.0.

Getting all IPv4 subnets...


<br>

### Subnets in Synergy Composer at https://syn420-dcs.hpedemo.local

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>networkId</th>
      <th>subnetmask</th>
      <th>domain</th>
      <th>gateway</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>10.48.0.0</td>
      <td>255.255.255.0</td>
      <td>hpedemo.local</td>
      <td>10.48.0.1</td>
    </tr>
    <tr>
      <td>10.48.10.0</td>
      <td>255.255.255.0</td>
      <td>hpedemo.local</td>
      <td>10.48.10.1</td>
    </tr>
    <tr>
      <td>192.168.10.0</td>
      <td>255.255.255.0</td>
      <td>None</td>
      <td>None</td>
    </tr>
    <tr>
      <td>192.168.20.0</td>
      <td>255.255.255.0</td>
      <td>None</td>
      <td>None</td>
    </tr>
  </tbody>
</table>

<br><br>
## Networks  <a name="Networks"></a>

In [177]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

# Loop over the addresses array and create subnets and ranges
for options in poc['networks']:
    
    options.update({
        "type" : "ethernet-networkV4",
        #"name": "IPv4Subnet"
    })
    # Remove/extract the association key since it does not belong there.
    association_option = options.pop('association', None)
    pprint(options)  
    
    try:
        print("Create ethernet network " + options['name'] )
        ethernet_network = ov.ethernet_networks.create(options)
    except HPOneViewException as e:
        print(e.msg)      
"""
options = {
    "name": "POC net",
    "vlanId": 3000,
    "ethernetNetworkType": "Tagged",
    "purpose": "General",
    "smartLink": False,
    "privateNetwork": False,
    "type" : "ethernet-networkV4",
    "connectionTemplateUri": None
}

ethernet_network = ov.ethernet_networks.create(options)
print("Created ethernet-network " + ethernet_network['name'] + "   URI: " + ethernet_network['uri'] + "\nsuccessfully.")
"""

{'name': 'VLAN2000_10.48.0',
 'purpose': 'Management',
 'type': 'ethernet-networkV4',
 'vlanId': 2000}
Create ethernet network VLAN2000_10.48.0
smartLink is required.


'\noptions = {\n    "name": "POC net",\n    "vlanId": 3000,\n    "ethernetNetworkType": "Tagged",\n    "purpose": "General",\n    "smartLink": False,\n    "privateNetwork": False,\n    "type" : "ethernet-networkV4",\n    "connectionTemplateUri": None\n}\n\nethernet_network = ov.ethernet_networks.create(options)\nprint("Created ethernet-network " + ethernet_network[\'name\'] + "   URI: " + ethernet_network[\'uri\'] + "\nsuccessfully.")\n'

<br><br>
## Delete the POC network 

In [12]:
ov.ethernet_networks.delete(ethernet_network)
print("delete network: " + ethernet_network['name'])

delete network: POC net


<br><br>
## Create bulk networks
How to set up multiple networks at once.

In [14]:
options_bulk = {
    "vlanIdRange": "30-40,50,100-109,200",
    "purpose": "General",
    "namePrefix": "Bulk-Ethernet",
    "smartLink": False,
    "privateNetwork": False,
    "type": "bulk-ethernet-networkV1",
    "bandwidth": {
        "maximumBandwidth": 10000,
        "typicalBandwidth": 2000
    }
}

# create the networks
ethernet_nets_bulk = ov.ethernet_networks.create_bulk(options_bulk)

# simply display whats exepected
for net in ethernet_nets_bulk: 
  print("created network: " + net['name'] + " URI: " + net['uri'])

created network: Bulk-Ethernet_30 URI: /rest/ethernet-networks/fbea227c-1096-465a-81fc-3589ff30a070
created network: Bulk-Ethernet_31 URI: /rest/ethernet-networks/75ecf717-2c2e-4484-882e-580ab7effbf3
created network: Bulk-Ethernet_32 URI: /rest/ethernet-networks/c486d648-9077-457e-a6c6-bac7f5c1829d
created network: Bulk-Ethernet_33 URI: /rest/ethernet-networks/7ccba565-96b1-42b1-8c2f-cd6cecf7d346
created network: Bulk-Ethernet_34 URI: /rest/ethernet-networks/72bf172a-88df-4f2d-9498-37f6326f60db
created network: Bulk-Ethernet_35 URI: /rest/ethernet-networks/2f710f0b-139d-4c6c-890f-6f229c9e822f
created network: Bulk-Ethernet_36 URI: /rest/ethernet-networks/55ae3a94-26c0-4640-8ff3-665e8ec0af24
created network: Bulk-Ethernet_37 URI: /rest/ethernet-networks/b5ccc270-20ac-4607-af59-1e612d243c4d
created network: Bulk-Ethernet_38 URI: /rest/ethernet-networks/86451626-6435-43fb-9927-4d9558a007c1
created network: Bulk-Ethernet_39 URI: /rest/ethernet-networks/98fb5ca5-6675-4ef7-9e9b-376cc7bc8bad


<br><br>
## Delete bulk network
As they are not needed for this demo

In [16]:
for net in ethernet_nets_bulk:
    ov.ethernet_networks.delete(net)
    print("delete network: " + net['name'])

delete network: Bulk-Ethernet_30
delete network: Bulk-Ethernet_31
delete network: Bulk-Ethernet_32
delete network: Bulk-Ethernet_33
delete network: Bulk-Ethernet_34
delete network: Bulk-Ethernet_35
delete network: Bulk-Ethernet_36
delete network: Bulk-Ethernet_37
delete network: Bulk-Ethernet_38
delete network: Bulk-Ethernet_39
delete network: Bulk-Ethernet_40
delete network: Bulk-Ethernet_50
delete network: Bulk-Ethernet_100
delete network: Bulk-Ethernet_101
delete network: Bulk-Ethernet_102
delete network: Bulk-Ethernet_103
delete network: Bulk-Ethernet_104
delete network: Bulk-Ethernet_105
delete network: Bulk-Ethernet_106
delete network: Bulk-Ethernet_107
delete network: Bulk-Ethernet_108
delete network: Bulk-Ethernet_109
delete network: Bulk-Ethernet_200


<br><br>
# Storage operations
## Show configured / existing storage (systems and pools)

In [56]:
storage_systems = ov.storage_systems.get_all()
storage_pools = ov.storage_pools.get_all()

# storage_pool_name = 'FC_r1'
# storage_pools = oneview_client.storage_pools.get_by('name', storage_pool_name)[0]

print("Storage Systems:")
if storage_systems:
    for stor in storage_systems: 
        print(stor['displayName'] + "    URI: " + stor['uri'])
else:
    print("None defined!")
    
print("\nStorage Pools:")
if storage_pools:
    for storpool in storage_pools:
        print(storpool['name'] + "    URI: " + stor['uri'])
else:
    print("None defined!")

Storage Systems:
None defined!

Storage Pools:
None defined!


In [57]:
if storpool is not None: 
    print (storpool)

NameError: name 'storpool' is not defined

<br><br>
## Add Storage Systems

<br><br>
## Show volume templates

In [31]:
storage_volume = ov.storage_volume_templates.get_all(filter="\"isRoot='False'\"")
if storage_volume:
    for storvol in storage_volume:
        print(storvol['name'])
else:
    print ("No storage volumes exists!")

No storage volumes exists!


<br><br>
## Create volume template

In [13]:
storage_pool_name = 'FC_r5'

# Get the storage pool by name to use in options
storage_pool = ov.storage_pools.get_by('name', storage_pool_name)[0]

# Gets the first Root Storage Volume Template available to use in options
root_template = ov.storage_volume_templates.get_all(filter="\"isRoot='True'\"")[0]
print(root_template['uri'])

options = {
    "name": "First Volume Template",
    "description": "",
    "rootTemplateUri": root_template['uri'],
    "properties": {
        "name": {
            "meta": {
                "locked": False
            },
            "type": "string",
            "title": "Volume name",
            "required": True,
            "maxLength": 100,
            "minLength": 1,
            "description": "A volume name between 1 and 100 characters"
        },
        "size": {
            "meta": {
                "locked": False,
                "semanticType": "capacity"
            },
            "type": "integer",
            "title": "Capacity",
            "default": 1073741824,
            "maximum": 17592186044416,
            "minimum": 268435456,
            "required": True,
            "description": "The capacity of the volume in bytes"
        },
        "description": {
            "meta": {
                "locked": False
            },
            "type": "string",
            "title": "Description",
            "default": "",
            "maxLength": 2000,
            "minLength": 0,
            "description": "A description for the volume"
        },
        "isShareable": {
            "meta": {
                "locked": False
            },
            "type": "boolean",
            "title": "Is Shareable",
            "default": False,
            "description": "The shareability of the volume"
        },
        "storagePool": {
            "meta": {
                "locked": False,
                "createOnly": True,
                "semanticType": "device-storage-pool"
            },
            "type": "string",
            "title": "Storage Pool",
            "format": "x-uri-reference",
            "required": True,
            "description": "A common provisioning group URI reference",
            "default": storage_pool['uri']
        },
        "snapshotPool": {
            "meta": {
                "locked": True,
                "semanticType": "device-snapshot-storage-pool"
            },
            "type": "string",
            "title": "Snapshot Pool",
            "format": "x-uri-reference",
            "default": storage_pool['uri'],
            "description": "A URI reference to the common provisioning group used to create snapshots"
        },
        "provisioningType": {
            "enum": [
                "Thin",
                "Full",
                "Thin Deduplication"
            ],
            "meta": {
                "locked": True,
                "createOnly": True
            },
            "type": "string",
            "title": "Provisioning Type",
            "default": "Thin",
            "description": "The provisioning type for the volume"
        }
    }
}

volume_template = oneview_client.storage_volume_templates.create(options)
print("Storage volume created...")

/rest/storage-volume-templates/bf62b10a-7a6c-4c7a-ba1d-a8c800809aaa
Storage volume created...


## Delete storage template
""" don´t use it, for the demo.. """

In [14]:
storage_volume = oneview_client.storage_volume_templates.get_all(filter="\"name='Roundtable Volume Template'\"")
for storvol in storage_volume:
    oneview_client.storage_volume_templates.delete(storvol)
    

## Show Enclosure / Server Hardware / Bay

In [15]:
print ("show enclosure group (enclosure_group_uri)")
enclosure = oneview_client.enclosure_groups.get_all()
for enc in enclosure:
    print(enc['name'] + " " + enc['uri'])

print("\nshow server hardware (server_hardware_type_uri)")
server_hardware_types = oneview_client.server_hardware_types.get_all(sort='name:descending')
# print(server_hardware_types)
for serverhw in server_hardware_types:
    # print(' %s ' % serverhw['model'])
    print(serverhw['model'] + " URI: " + serverhw['uri'] )

print("\nEnclosure/Bay (server_hardware_uri)")   
server_hardware = oneview_client.server_hardware.get_all()
for server in server_hardware:
    print(server['name'] + " " + server['model'] + " " + server['uri'])

show enclosure group (enclosure_group_uri)
SY 3 Frame SAS FC VC /rest/enclosure-groups/73668dd8-d2cf-4004-b7df-8ca7cfeabbc0
SY 1 Frame VC /rest/enclosure-groups/92e527ff-7129-489f-87dc-e7d5821f13c6
SY 3 Frame SAS VC /rest/enclosure-groups/109eed5b-4cff-470e-b2ca-ffb26eb5c678

show server hardware (server_hardware_type_uri)
Synergy 660 Gen9 URI: /rest/server-hardware-types/A435F2DD-9EB9-4F98-B854-1A6B35140DC9
Synergy 660 Gen10 URI: /rest/server-hardware-types/7AA9D8EE-C46A-4901-8D10-C252AF9DB1A4
Synergy 480 Gen9 URI: /rest/server-hardware-types/860CAF19-0D67-401A-A1D2-536AA9356102
Synergy 480 Gen9 URI: /rest/server-hardware-types/1131E4D0-9C51-4615-A20E-CCF00F02A94F
Synergy 480 Gen10 URI: /rest/server-hardware-types/77E175FB-A4AA-4E95-8780-D1E9A40D3887
Synergy 480 Gen10 URI: /rest/server-hardware-types/56C7D5F1-6221-4ACB-938B-21916B7DE5F3

Enclosure/Bay (server_hardware_uri)
CTC Synergy HE01, bay 1 Synergy 480 Gen10 /rest/server-hardware/39313738-3134-5A43-4A37-343030373531
CTC Synergy 

<br>
## Show Serverprofile


In [16]:
print("show server profile templates:")
all_srv_templates = oneview_client.server_profile_templates.get_all()
for srv_tmp in all_srv_templates:
    print(srv_tmp['name'] + "   URI:   " + srv_tmp['uri'])

print("\nshow server profiles:")
all_profiles = oneview_client.server_profiles.get_all()
for profile in all_profiles:
    print(profile['name']+ "   URI:  " + srv_tmp['uri'])

# my_profile = oneview_client.server_profiles.get_by_name("Roundtable - API Demo Template (DirkD)")

show server profile templates:
SRT-Storage-BfS-W2k16   URI:   /rest/server-profile-templates/1df43d4a-76c3-4821-bff3-729446dc7231
Roundtable - Python (DirkD)   URI:   /rest/server-profile-templates/1e2c29f5-e146-42d4-9f78-a057b41d3953
SRT-LC-WIN-TP   URI:   /rest/server-profile-templates/308e1b0a-2e7c-40aa-bdf9-84eb87b42548
SRT-StorageDemo-ESX   URI:   /rest/server-profile-templates/534d2abd-989c-456e-a84b-8c112ce3fcac
Demo-CTC Windows2016-DC-byI3s   URI:   /rest/server-profile-templates/7a5d77bc-a372-4a44-8a52-b538ba703b97
SRT-Storage-BfS-ESX   URI:   /rest/server-profile-templates/823883ee-0f0e-4554-8e00-a909001b28af
Demo-CTC Windows2016-DC-byI3s_Gen10   URI:   /rest/server-profile-templates/894f818a-9604-48cb-ab9f-ea6c240a9381
DEMO-CTC ESXi Server Template   URI:   /rest/server-profile-templates/ac2995c1-8b21-482b-80e8-7157f1984b75
SRT-LC-WIN BfS   URI:   /rest/server-profile-templates/b18b3cf9-8f47-4a6a-be35-18bba3078acd
DEMO-CTC Stratoscale Template   URI:   /rest/server-profile-t

<br><br>
## Create server profile (takes 2-3 minutes)

In [17]:
powerOn = {
    "powerState": "On",
    "powerControl": "MomentaryPress"
}

powerOff = {
    "powerState": "Off",
    "powerControl": "PressAndHold"
}

#template_name = "Roundtable - API Demo Template (DirkD)"
template_name = "Roundtable - Python (DirkD)"
server_name = "Roundtable - API Demo Server (DirkD)"

# server_hardware_uri = '/rest/server-hardware/39313738-3133-5A43-4A37-343030373931' # SY660 G10
server_hardware_uri = '/rest/server-hardware/39313738-3134-5A43-4A37-343030373536' # SY480
server_template_uri = oneview_client.server_profile_templates.get_by_name(template_name)

server_power = oneview_client.server_hardware.update_power_state(powerOff, server_hardware_uri) # turn off server

try:
    print ("create server profile")
    roundtable_server = oneview_client.server_profile_templates.get_new_profile(server_template_uri['uri'])
    roundtable_server["name"] = server_name
    roundtable_server["serverHardwareUri"] = server_hardware_uri

    for name in roundtable_server["osDeploymentSettings"]["osCustomAttributes"]:
        if (name['name'] == "NewUser"):
            name['value']= "dirk"
        if (name['name'] == "NewUserPassword"):
            name['value']= "HalloRoundtable!"   
    profile = oneview_client.server_profiles.create(roundtable_server)

    
except:
    print(server_name + " Server already exists")

server_power = oneview_client.server_hardware.update_power_state(powerOn, server_hardware_uri) # turn on server


create server profile


### Show individual parameters from Image (Imagestreamer)

In [34]:
    #print ("create server profile")
    my_server = ov.server_profile_templates.get_new_profile(server_template_uri['uri'])
    my_server["name"] = server_name
    my_server["serverHardwareUri"] = server_hardware_uri

    for name in my_server["osDeploymentSettings"]["osCustomAttributes"]:
        print(name['name'] + ": " + str(name['value']))
   


NameError: name 'server_template_uri' is not defined

## Delete server profile
!!! do not use, if not necessary !!!

In [ ]:
server_power = oneview_client.server_hardware.update_power_state(powerOff, server_hardware_uri) # turn on server
oneview_client.server_profiles.delete(profile)

### example: how to get the values from json

In [ ]:
    server_hardware = oneview_client.server_hardware.get_all()
   # print(server_hardware)
    
    for server in server_hardware:
        for ports in server['portMap']['deviceSlots']:
            for mac in ports['physicalPorts']: 
                for wwnn in mac['virtualPorts']:
                    print(server['name']+ " model:" + server['model'] + " " + str(server['memoryMb']) + " MB mac: " + str(mac['mac']) + " wwnn: " + str(wwnn['wwnn'])) 

## Application deployment


Webserver NGINX running on docker.

##### What we need:

IP address of deployed system.

<br>
<img src="Pictures/NGINX.png" height="500" width="500" align="left">


In [21]:
import os
import paramiko
import time

ssh = paramiko.SSHClient()

server_name = "Roundtable - API Demo Server (DirkD)"
username = 'root'
password = 'Compaq1!'

deployed_server = oneview_client.server_profiles.get_by_name(server_name)

for name in deployed_server["osDeploymentSettings"]["osCustomAttributes"]:
    if (name['name'] == "Team0NIC1.ipaddress"):
        ip_address = name['value']

        
print("Login with user: " + username + " Server:" + ip_address)        
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())         # add unknown Host-Keys
ssh.connect(ip_address, username=username, password=password)     # login
ssh.exec_command('docker run -d --name nginx -p 80:80 nginx')
#time.sleep(10)
stdin, stdout, stderr = ssh.exec_command("docker exec -it nginx sed -i '\''s/nginx/the Synergy Roundtable/g'\'' /usr/share/nginx/html/index.html", get_pty=True)
print("http://" + ip_address)
# print(stdout.read())
# print(stderr.read())


Login with user: root Server:10.0.20.85
http://10.0.20.85


### Excel Export

xlswriter:
https://xlsxwriter.readthedocs.io/

example: we extrace some data to excel XLS

In [32]:
import xlsxwriter
workbook = xlsxwriter.Workbook('roundtable.xlsx')
worksheet = workbook.add_worksheet()

# Add a bold format to use to highlight cells.
bold = workbook.add_format({'bold': True})

# Text with formatting.
worksheet.write(0,0, 'Synergy Roundtable', bold)

# Start from the first cell below the headers.
row = 4
worksheet.write(row, 0, "Servername", bold)
worksheet.write(row, 1, "Model", bold)
worksheet.write(row, 2, "Memory", bold)
worksheet.write(row, 3, "MAC address", bold)
worksheet.write(row, 4, "WWN address", bold)
worksheet.write(row, 5, "Status", bold)
row += 1

server_hardware = oneview_client.server_hardware.get_all()
#print(server_hardware)

for server in server_hardware:
 col = 0
 for ports in server['portMap']['deviceSlots']:
     for mac in ports['physicalPorts']: 
         for wwnn in mac['virtualPorts']:
             # print(server['name']+ " model:" + server['model'] + " " + str(server['memoryMb']) + " MB mac: " + str(mac['mac']) + " wwnn: " + str(wwnn['wwnn']))
             worksheet.write(row,col, server['name'])
             worksheet.write(row,col+1, server['model'])
             worksheet.write(row,col+2, server['memoryMb'])
             worksheet.write(row,col+3, mac['mac'])
             worksheet.write(row,col+4, wwnn['wwnn'])
             worksheet.write(row,col+5, server['status'])
             row += 1
workbook.close()
print ('Excel File roundtable.xlsx created')

ModuleNotFoundError: No module named 'xlsxwriter'